In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130320

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,109177.9491
2018-02-28,114371.2658
2018-03-31,135405.3074
2018-04-30,103041.2569
2018-05-31,111993.8575
2018-06-30,111011.7819
2018-07-31,128128.3378
2018-08-31,123941.4700
2018-09-30,108679.0023


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    109177.9491
 2018-02-28    114371.2658
 2018-03-31    135405.3074
 2018-04-30    103041.2569
 2018-05-31    111993.8575
 2018-06-30    111011.7819
 2018-07-31    128128.3378
 2018-08-31    123941.4700
 2018-09-30    108679.0023
 2018-10-31    131790.5595
 2018-11-30    117242.4327
 2018-12-31    135823.3864
 2019-01-31    200988.2830
 2019-02-28    152494.9900
 2019-03-31    172389.8605
 2019-04-30    143567.2002
 2019-05-31    148644.2630
 2019-06-30    127440.6243
 2019-07-31    157077.6091
 2019-08-31    165666.8963
 2019-09-30    142218.9175
 2019-10-31    130917.9348
 2019-11-30    123283.5397
 2019-12-31    123219.0519
 2020-01-31    136894.1848
 2020-02-29    112243.6441
 2020-03-31    145688.8033
 2020-04-30     95527.0519
 2020-05-31    108539.6538
 2020-06-30    113982.9375
 2020-07-31    115803.1523
 2020-08-31    108523.2395
 2020-09-30     94686.8497
 2020-10-31    112330.1143
 2020-11-30    108511.6510
 2020-12-31    101620.2326

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.969391
p-value : 0.300153
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -10.802017
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=965.698, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=960.935, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=958.068, Time=0.03 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=959.196, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=957.367, Time=0.12 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=958.887, Time=0.17 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=953.085, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=950.904, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=951.976, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=954.038, Time=0.07 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=959.495, Time=0.10 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=949.278, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=951.145, Time=0.06 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=953.264, Time=0.11 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=955.593, Time=0.12 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -470.622
Method:                       css-mle   S.D. of innovations          17661.446
Date:                Sat, 11 Sep 2021   AIC                            949.243
Time:                        04:35:35   BIC                            956.194
Sample:                             0   HQIC                           951.791
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.222e+05   8430.978     14.500      0.000    1.06e+05    1.39e+05
ar.L1.y        0.3441      0.141      2.442      0.015       0.068       0.620
ar.L2.y        0.3551      0.141      2.521      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([115767.85798755, 115184.22324236]), array([17661.44559181, 18677.74392003]), array([[ 81152.0607127 , 150383.65526241],
       [ 78576.51784664, 151791.92863809]]))
